In [18]:
import os 
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ["OPENAI_API_KEY"]

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [ ]:
import asyncio
from typing import List
from pydantic import BaseModel
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage

system_template = """
Giả sử bạn là chuyên gia bác sĩ TamAnhHospital , lưu ý trả lời bằng tiếng việt , những câu hỏi không liên quan đến sức khỏe sẽ không được trả lời.
Kịch bản trả lời : 
Chào khách hàng (tên khách hàng) , tôi là bác sĩ TamAnhHospital , tôi xin phép trả lời câu hỏi của anh chị như sau .
Giải đáp thắc mắc:
xác định vấn đề.
Đưa ra giải pháp.
Đưa ra lời khuyên.
Đề nghị người hỏi đến bệnh viện của TamAnhHospital để được tư vấn cụ thể hơn.

Sau đây là kiến thức được cung cấp:
-----------------------------------------
{content}

"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
class CustomState(BaseModel):
    messages: List[BaseMessage]
    content: str

async def call_model(state: CustomState):
    chain = prompt | llm
    response = chain.invoke({'messages': state.messages, 'content': state.content})
    return {"messages": response}

workflow = StateGraph(state_schema=CustomState)

workflow.add_edge(START, "llm")
workflow.add_node("llm", action=call_model)

app = workflow.compile(checkpointer=MemorySaver())

config = {"configurable": {"thread_id": "abc345"}}
async def main():
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            print("Exiting...")
            break
        
        content = "Đây là thông tin tham khảo về y học từ bệnh viện TamAnhHospital."
        input_messages = [HumanMessage(content=query)]
        output = await app.ainvoke(
            {"messages": input_messages, "content": content}, 
            config=config
        )
        
        for message in output["messages"]:
            print(f"TamAnhHospital Bot: {message.content}")
await main()


AttributeError: 'tuple' object has no attribute 'content'